In [ ]:
import ipywidgets as ipw
from home.app_store import AiidaLabAppStore

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
display(ipw.HTML("""/<a href="start.ipynb"> home </a>/ appstore"""))


# <center>App Store</center>

In [ ]:
control = AiidaLabAppStore()
display(control)
display(ipw.HBox([control.items_per_page, control.page_selector]))
display(control.render())
display(ipw.HBox([control.items_per_page, control.page_selector]))